PREPROCESSING

In [1]:
%load_ext autoreload
%autoreload 2

In [48]:
import os
import autogen
import autogen.runtime_logging
import pandas as pd
import sqlite3
import json
import docx
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
from preprocessing.preprocessing import score_processor
from preprocessing.preprocessing import generate_conversations,process_pdf,rubric_processor
from multi_agent import subjective_agent,objective_agent,assessment_plan_agent
from general_SOAP_Note_agent import single_agent_generation,soapnoteagent
from evaluation_agent import evaluator_agent,score_generation,detail_score_generation
from refiner_agent import refiner_agent,refine_generation

Patching name='__init__', member=<function LLMLingua.__init__ at 0x000001797F2EDA80>, patched=<function function.__call__ at 0x000001797F2ED9E0>
Patching name='compress_text', member=<function LLMLingua.compress_text at 0x000001797F2EDB20>, patched=<function function.__call__ at 0x000001797F2EDC60>


In [4]:
conversation_list,conversation_names=generate_conversations('C:/Users/rahul/Desktop/SOAPNote/dataset/VAT Patient Transcripts/VAT Patient Transcripts/')
demo_soap_note=process_pdf("C:/Users/rahul/Desktop/SOAPNote/dataset/demo_sample/SOAP note- M.A..pdf")
demo_conversation=conversation_list[7]
rubric=rubric_processor('C:/Users/rahul/Desktop/SOAPNote/rubric/rubric.docx')
s_rubric=rubric.split('\nSubjective\n')[1].split('\nObjective\n')[0]
o_rubric=rubric.split('\nObjective\n')[1].split('\nAssessment\n')[0]
ap_rubric=rubric.split('\nAssessment\n')[1].split('\nIssues of Concern\n')[0]

In [ ]:
import os
from docx import Document

doc=Document()



SINGLE-AGENT GENERATION

In [31]:
single_soap_note_list=[]
for conversation in conversation_list:
    soap_note=single_agent_generation(conversation,rubric,demo_soap_note)
    single_soap_note_list.append(soap_note)

In [32]:
len(single_soap_note_list)

28

MULTI-AGENT GENERATION

In [33]:
s_agent = subjective_agent(s_rubric)
o_agent = objective_agent(o_rubric)
ap_agent = assessment_plan_agent(ap_rubric)
assistant_id = os.environ.get("ASSISTANT_ID", None)
config_list = config_list_from_json("C:/Users/rahul/Documents/OAI_CONFIG_LIST.json")
llm_config = {"config_list": config_list}
user_proxy = UserProxyAgent(
        name="user_proxy",
        code_execution_config={
            "work_dir": "coding",
            "use_docker": False,
        },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
        is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
        human_input_mode="NEVER",
        max_consecutive_auto_reply=0,
)
groupchat = autogen.GroupChat(
    agents=[user_proxy,s_agent, o_agent, ap_agent],
    messages=[],
    max_round=4
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config
)
for i in range(len(conversation_list)):
    logging_session_id=autogen.runtime_logging.start(config={"dbname":f"full_split_conversation_mini{i}.db"})
    conversation=conversation_list[i]
    result=user_proxy.initiate_chat(
        manager,
        message="Generate a comprehensive SOAP note based strictly on the following conversation between a patient and a clinician:"
                + conversation +
                "\nEnsure the SOAP note includes all relevant details from the conversation while adhering to the structure and style of the provided demo: " +demo_soap_note +
                "\nEach agent should contribute according to their role and rubric."
    )
    autogen.runtime_logging.stop()

user_proxy (to chat_manager):

Generate a comprehensive SOAP note based strictly on the following conversation between a patient and a clinician:Speaker 1: Patient L.V.G.
Speaker 2: Clinician
Speaker 3: Patient L.V.G.’s Alexa

EBVTT
-------------------------------------------------- 1:1 Interview ---------------------------------------------- 
00:00:02.930 --> 00:00:05.360
Speaker 1: Yes, when they say tell me all the nouns or the words you can say with a letter S in 1 min, I can't come up with more than 4 or 5.
 
00:00:20.580 --> 00:00:21.740
Speaker 2: Got you. Okay.
 
00:00:21.740 --> 00:00:22.545
Speaker 1: That's That's how Anania shows up and
 
00:00:29.940 --> 00:00:30.480
Speaker 2: Yeah.
 
00:00:30.480 --> 00:00:36.692
Speaker 1: I play bridge online with my husband, and we have one something. We do, all right. and I don't have any cognitive disabilities yet, but I'm struggling to speak. And That's it.
 
00:00:57.770 --> 00:01:09.229
Speaker 2: Gotcha. Okay? Alright. Do you? G

In [34]:
from preprocessing.preprocessing import conversation_db_processor
split_soap_note_list=[]
for i in range(len(conversation_list)):
    soap_notes_data=sqlite3.connect(f'C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/full_split_conversation_mini{i}.db')
    soap_note=conversation_db_processor(soap_notes_data)
    split_soap_note_list.append(soap_note)

In [36]:
print(split_soap_note_list[0])


Subjective (S):

Chief Complaint (CC): The primary issue as stated by the patient, L.V.G., is difficulty with speech, specifically characterized by challenges in recalling words during communication tasks such as naming nouns that start with the letter 'S.' The patient does not report any current cognitive disabilities but expresses struggles with communication and speech.

History of Present Illness (HPI): L.V.G., an 81-year-old female, is presenting with difficulties in speech, specifically word-finding issues. The onset of these symptoms is unknown, but she has engaged in speech therapy in January of the previous year, which was described as unhelpful. The therapy focused on improving anomia through worksheets, yet the patient noted no improvement. She does not currently attend speech therapy and has not found prior interventions effective. During interactions, she experiences more difficulty with certain word retrieval tasks like items beginning with a specific letter and lists th

In [37]:
print(single_soap_note_list[0])

**SOAP Note for Patient L.V.G.**

**Subjective (S):**  
Patient L.V.G., an 81-year-old female, reported difficulties in verbal communication, particularly with naming tasks involving specific letters and recalling words/phrases. She does not have any diagnosed cognitive disabilities affecting reading and writing but mentions struggles with word retrieval (anomia). The patient has previously undergone speech therapy for anomia without significant improvement and is not currently receiving speech therapy. Patient expresses frustration with being understood on the phone, citing impatience from her conversational partners due to her aphasia. She typically communicates with her husband and engages in activities like playing bridge online and watercolor painting, which she manages without cognitive difficulty. The patient is open to learning and utilizing technology to aid communication and daily tasks.

**Objective (O):**  
A series of cognitive communication assessments were conducted:
- *

EVALUATION AND FINAL LIST

In [38]:
split_evaluation_list=[]
single_evaluation_list=[]
final_soap_note_list=[]
for i in range(len(conversation_list)):
    soap_note=single_soap_note_list[i]
    split_soap_note=split_soap_note_list[i]
    conversation=conversation_list[i]
    split_evaluation=score_generation(split_soap_note,conversation,rubric,demo_conversation,demo_soap_note)
    single_evaluation=score_generation(soap_note,conversation,rubric,demo_conversation,demo_soap_note)
    try:
        if(score_processor(split_evaluation) and score_processor(single_evaluation)):
            if(score_processor(split_evaluation)>score_processor(single_evaluation)):
                final_soap_note_list.append(split_soap_note)
            elif(score_processor(split_evaluation)<score_processor(single_evaluation)):
                final_soap_note_list.append(soap_note)
            else:
                if(len(split_soap_note)>len(soap_note)):
                    final_soap_note_list.append(split_soap_note)
                else:
                    final_soap_note_list.append(soap_note)
    except Exception as e:
        print(split_evaluation)
        print(single_evaluation)
    split_evaluation_list.append(split_evaluation)
    single_evaluation_list.append(single_evaluation)





In [39]:
len(final_soap_note_list)

28

In [40]:
final_soap_note_list

['**SOAP Note for Patient L.V.G.**\n\n**Subjective (S):**  \nPatient L.V.G., an 81-year-old female, reported difficulties in verbal communication, particularly with naming tasks involving specific letters and recalling words/phrases. She does not have any diagnosed cognitive disabilities affecting reading and writing but mentions struggles with word retrieval (anomia). The patient has previously undergone speech therapy for anomia without significant improvement and is not currently receiving speech therapy. Patient expresses frustration with being understood on the phone, citing impatience from her conversational partners due to her aphasia. She typically communicates with her husband and engages in activities like playing bridge online and watercolor painting, which she manages without cognitive difficulty. The patient is open to learning and utilizing technology to aid communication and daily tasks.\n\n**Objective (O):**  \nA series of cognitive communication assessments were conduc

EVALUATION-REFINEMENT LOOP

In [41]:
final_list=[]
complete_evaluation=[]
for i in range(len(conversation_list)):
    soap_note=final_soap_note_list[i]
    conversation=conversation_list[i]
    curr_score=0
    curr_rounds=0
    evaluation_list=[]
    print(f"Processing conversation {i+1}/{len(conversation_list)}")
    while(curr_score<9 and curr_rounds<10):
        print(f"Round {curr_rounds + 1}")
        evaluation=score_generation(soap_note,conversation,rubric,demo_conversation,demo_soap_note)
        print(evaluation)
        evaluation_list.append(evaluation)
        curr_score=score_processor(evaluation)
        if(curr_score>=9):
            print(f"Final score: {curr_score}")
            final_list.append(soap_note)
            break
        refined_soap_note=refine_generation(soap_note,conversation,evaluation)
        if refined_soap_note and refined_soap_note.strip()!=soap_note.strip():
            soap_note=refined_soap_note
        curr_rounds=curr_rounds+1
    if curr_rounds >= 10 and curr_score < 9:
        print(f"Max rounds ({10}) reached. Final score: {curr_score}. Adding best attempt to final list.")
        final_list.append(soap_note)
    print(f"Completed conversation {i+1}. Final score: {curr_score}\n")
    complete_evaluation.append(evaluation_list)


Processing conversation 1/28
Round 1
### EXTRACTION PHASE

**Key Information Points from Conversation:**
1. L.V.G. has difficulties with verbal communication, particularly with letter-specific tasks.
2. L.V.G. mentioned having anomia.
3. Patient's previous speech therapy did not help.
4. L.V.G. plays bridge online and paints.
5. Struggles to be understood on the phone due to aphasia.
6. Engages in daily tasks independently.
7. Has multiple doctors due to health issues.
8. Participated in verbal repetition tasks with some success.
9. Participated in numeration and categorization tasks.
10. Uses Alexa and desires to utilize technology for daily tasks.

### VERIFICATION PHASE

**Check each point against the SOAP note:**
1. Verbal communication difficulties noted. (Present)
2. Anomia mentioned. (Present)
3. Previous speech therapy mentioned as ineffective. (Present)
4. Activities such as online bridge and painting noted. (Present)
5. Phone communication struggles due to aphasia noted. (Pre

In [42]:
from docx import Document
output_folder="C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/final_soap_note"
for i in range(len(conversation_list)):
    doc=Document()
    doc.add_paragraph(final_list[i])
    filename=f"{conversation_names[i]}.docx"
    filepath = os.path.join(output_folder, filename)
    doc.save(filepath)
    print(f"Created: {filepath}")


Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/final_soap_note\07-17-2024 L.V.G Baseline.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/final_soap_note\07-18-2024 (Week 1) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/final_soap_note\07-25-2024 (Week 2) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/final_soap_note\08-01-2024 (Week 3) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/final_soap_note\08-08-2024 (Week 4) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/final_soap_note\08-22-2024 (Week 6) LVG Debrief Interview.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/final_soap_note\07-18-2024 (Week 0) M.A. Baseline.docx.docx
Created: C:/Users/rah

In [43]:
from docx import Document
output_folder="C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_agent_soap_note"
for i in range(len(conversation_list)):
    doc=Document()
    doc.add_paragraph(single_soap_note_list[i])
    filename=f"{conversation_names[i]}.docx"
    filepath = os.path.join(output_folder, filename)
    doc.save(filepath)
    print(f"Created: {filepath}")

Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_agent_soap_note\07-17-2024 L.V.G Baseline.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_agent_soap_note\07-18-2024 (Week 1) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_agent_soap_note\07-25-2024 (Week 2) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_agent_soap_note\08-01-2024 (Week 3) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_agent_soap_note\08-08-2024 (Week 4) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_agent_soap_note\08-22-2024 (Week 6) LVG Debrief Interview.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_agent_soap_note\07-18-2024 (Week

In [44]:
output_folder="C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/multi_agent_soap_note"
for i in range(len(conversation_list)):
    doc=Document()
    doc.add_paragraph(split_soap_note_list[i])
    filename=f"{conversation_names[i]}.docx"
    filepath = os.path.join(output_folder, filename)
    doc.save(filepath)
    print(f"Created: {filepath}")

Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/multi_agent_soap_note\07-17-2024 L.V.G Baseline.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/multi_agent_soap_note\07-18-2024 (Week 1) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/multi_agent_soap_note\07-25-2024 (Week 2) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/multi_agent_soap_note\08-01-2024 (Week 3) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/multi_agent_soap_note\08-08-2024 (Week 4) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/multi_agent_soap_note\08-22-2024 (Week 6) LVG Debrief Interview.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/multi_agent_soap_note\07-18-2024 (Week 0) M.A

In [45]:
output_folder="C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/evaluation_notes"
for i in range(len(conversation_list)):
    doc=Document()
    doc.add_paragraph(complete_evaluation[i])
    filename=f"{conversation_names[i]}.docx"
    filepath = os.path.join(output_folder, filename)
    doc.save(filepath)
    print(f"Created: {filepath}")

Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/evaluation_notes\07-17-2024 L.V.G Baseline.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/evaluation_notes\07-18-2024 (Week 1) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/evaluation_notes\07-25-2024 (Week 2) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/evaluation_notes\08-01-2024 (Week 3) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/evaluation_notes\08-08-2024 (Week 4) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/evaluation_notes\08-22-2024 (Week 6) LVG Debrief Interview.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/evaluation_notes\07-18-2024 (Week 0) M.A. Baseline.docx.docx
Created: C:/Us

In [46]:
output_folder="C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/split_evaluation_notes"
os.makedirs(output_folder)
for i in range(len(conversation_list)):
    doc=Document()
    doc.add_paragraph(split_evaluation_list[i])
    filename=f"{conversation_names[i]}.docx"
    filepath = os.path.join(output_folder, filename)
    doc.save(filepath)
    print(f"Created: {filepath}")

Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/split_evaluation_notes\07-17-2024 L.V.G Baseline.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/split_evaluation_notes\07-18-2024 (Week 1) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/split_evaluation_notes\07-25-2024 (Week 2) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/split_evaluation_notes\08-01-2024 (Week 3) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/split_evaluation_notes\08-08-2024 (Week 4) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/split_evaluation_notes\08-22-2024 (Week 6) LVG Debrief Interview.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/split_evaluation_notes\07-18-2024 (Week

In [47]:
output_folder="C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_evaluations"
os.makedirs(output_folder)
for i in range(len(conversation_list)):
    doc=Document()
    doc.add_paragraph(single_evaluation_list[i])
    filename=f"{conversation_names[i]}.docx"
    filepath = os.path.join(output_folder, filename)
    doc.save(filepath)
    print(f"Created: {filepath}")

Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_evaluations\07-17-2024 L.V.G Baseline.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_evaluations\07-18-2024 (Week 1) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_evaluations\07-25-2024 (Week 2) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_evaluations\08-01-2024 (Week 3) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_evaluations\08-08-2024 (Week 4) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_evaluations\08-22-2024 (Week 6) LVG Debrief Interview.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o-mini/single_evaluations\07-18-2024 (Week 0) M.A. Baseline.docx.docx


In [29]:
from pypdf import PdfReader
file_name='C:/Users/rahul/Desktop/SOAPNote/multi_stage_classification.pdf'
reader = PdfReader(file_name)
demo_text=''
for i in range(0,len(reader.pages)):
    page=reader.pages[i]
    text=page.extract_text()
    text=text.replace('\n','')
    demo_text+=text
demo_text

' Code  Name:- No  response  Description:- Remaining  silent  without  a  verbal  response   Example:-  Alexa:  “What  can  I  help  you  with  today?”  Client:  (remains  silent—no  “Hey  Assistant,”  no  follow ‑ up.)  @No  response  Alexa(after  timeout):  “Are  you  still  there?  How  can  I  assist?”    Code  Name: -Articulation  error  Description:- Mispronunciation  of  words,  phrases,  and  sentences  Example:-  Client:  “Set  a  ti ‑ myer  for  five  minutes.”  @Articulation  error  Alexa:  “Sorry,  I  didn’t  catch  that.  Did  you  mean  ‘timer  for  five  minutes’?”    Code  Name:- Semantic  error  Description:- Misuse  of  words,  phrases  Example:-  Client:  “Assistant,  set  a  roof  for  7\u2009AM  tomorrow.” @Semantic  error  Alexa:“I’m  not  sure  what  a  ‘roof’  is  in  this  context.  Did  you  mean  ‘alarm’?”    Code  Name:- Syntactic  error  Description:- Misuse  of  syntax  Example:-  Client:  “Lights  kitchen  in  the  turn  off.” @Syntactic error  Alexa:  “H

In [41]:
from classifier_agents import multi_class_classification
classification_results=[]
for conversation in conversation_list:
    classification_results.append(multi_class_classification(conversation,demo_text))

In [43]:
for result in classification_results:
    print(result)

In the provided conversation transcript, there are no direct or indirect interactions between the patient (Speaker 1) and Alexa (Speaker 3) that result in a communication breakdown as per the given categories. The conversation primarily involves Speaker 1 and Speaker 2 discussing various tasks and capabilities related to using Alexa, but there are no instances where Speaker 1 directly interacts with Alexa and experiences a breakdown in communication. Therefore, no classification is required for this transcript.
Based on the provided conversation transcript, there are no direct or indirect interactions between the patient (Speaker 3) and Alexa. The conversation is primarily between the clinician, the graduate student, and the patient discussing the patient's use of technology and potential use of Alexa. Since there is no actual interaction with Alexa, there are no instances of communication breakdown to classify according to the given categories. Therefore, no classification is required

In [47]:
from docx import Document
output_folder="C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns"
for i in range(len(classification_results)):
    doc=Document()
    doc.add_paragraph(classification_results[i])
    filename=f"{conversation_names[i]}.docx"
    filepath = os.path.join(output_folder, filename)
    doc.save(filepath)
    print(f"Created: {filepath}")

Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns\07-17-2024 L.V.G Baseline.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns\07-18-2024 (Week 1) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns\07-25-2024 (Week 2) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns\08-01-2024 (Week 3) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns\08-08-2024 (Week 4) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns\08-22-2024 (Week 6) LVG Debrief Interview.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns\07-18-2024 (Week 0) M.A. Baseline.doc

In [34]:
print(classification_results[2])

Based on the conversation transcript provided, here are the instances of communication breakdown between the patient (Speaker 1) and Alexa (Speaker 4), along with their respective categories:

1. **Instance 1:**
   - **Conversation Excerpt:**
     - Speaker 1: "Speaker 4, change your wake up word. It didn't do anything."
     - Speaker 4: (No response to the command)
   - **Category:** @Alexa error
   - **Reason:** The verbal command from the client was clear, but Alexa did not respond appropriately, indicating a possible speech recognition error or other issue.

2. **Instance 2:**
   - **Conversation Excerpt:**
     - Speaker 1: "Speaker 4, play sweet Caroline."
     - Speaker 4: "Sweet Caroline and similar song on Amazon Music."
     - Speaker 1: "Speaker 4, raise the volume. Speaker 4 stop. Pause. Speaker 4, go back. Go back to the beginning. Speaker 4, play sweet Caroline."
     - Speaker 4: "Sweet Caroline and similar songs on Amazon Music."
   - **Category:** @Alexa error
   - **

In [35]:
conversation_names[2]

'07-25-2024 (Week 2) LVG User Needs Assessment.docx'

In [1]:
import os
import docx
single_soap_note_list=[]
multi_soap_note_list=[]
final_soap_note_list=[]
conversation_breakdown_list=[]
path="C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/single_agent_soap_note"
for i in os.listdir(path):
    file=path+'/'+i
    doc=docx.Document(file)
    for j in doc.paragraphs:
        print(j.text)
        single_soap_note_list.append(j.text)
path="C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/multi_agent_soap_note"
for i in os.listdir(path):
    file=path+'/'+i
    doc=docx.Document(file)
    for j in doc.paragraphs:
        print(j.text)
        multi_soap_note_list.append(j.text)
path="C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/final_soap_note"
for i in os.listdir(path):
    file=path+'/'+i
    doc=docx.Document(file)
    for j in doc.paragraphs:
        print(j.text)
        final_soap_note_list.append(j.text)
path="C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns"
for i in os.listdir(path):
    file=path+'/'+i
    doc=docx.Document(file)
    for j in doc.paragraphs:
        print(j.text)
        conversation_breakdown_list.append(j.text)



**SOAP Note**

**Subjective (S):**
M.A. attended the virtual session accompanied by his mother, who is one of his primary caregivers assisting him with Activities of Daily Living (ADLs) following his Traumatic Brain Injury (TBI) that resulted in blindness and cognitive communication deficits. M.A. was cooperative and pleasant throughout the session, engaging willingly with the interview questions for the user needs assessment and participating in the training tasks. His mother provided additional details to elaborate on M.A.'s responses. M.A. relies on his mother for setting up appointments and reminders, although he uses Siri on his iWatch and iPhone for scheduling. He requires reminders for regular therapy appointments at BRAIN and Loma Linda, as well as for medical and dental appointments.

**Objective (O):**
A user needs assessment was conducted using an interview-style Q&A to determine M.A.'s needs related to scheduling and reminders. M.A. is caregiver-dependent for setting up app

In [6]:
len(conversation_breakdown_list)

28

In [7]:
split_evaluation_list=[]
single_evaluation_list=[]
final_evaluation_list=[]
for i in range(len(conversation_list)):
    soap_note=single_soap_note_list[i]
    split_soap_note=multi_soap_note_list[i]
    final_soap_note=final_soap_note_list[i]
    conversation_breakdown=conversation_breakdown_list[i]
    conversation=conversation_list[i]
    single_evaluation=detail_score_generation(soap_note,conversation,conversation_breakdown)
    split_evaluation=detail_score_generation(split_soap_note,conversation,conversation_breakdown)
    final_evaluation=detail_score_generation(final_soap_note,conversation,conversation_breakdown)
    single_evaluation_list.append(single_evaluation)
    split_evaluation_list.append(split_evaluation)
    final_evaluation_list.append(final_evaluation)


In [13]:
final_evaluation_list

["The SOAP note provided does not include any specific breakdown instances from the conversation between the patient and Alexa. The note describes the patient's interaction with the clinician and the use of technology, but it does not provide a transcript or detailed analysis of communication breakdowns with Alexa. Therefore, based on the criteria of identifying and classifying communication breakdowns according to the categories provided, the SOAP note scores 0 out of 10.",
 'The SOAP note provided does not mention any breakdown instances related to the interaction between the patient and Alexa, as specified in the breakdown instances from the conversation. The entire conversation is between the clinician and the patient, with no interaction involving Alexa or any other virtual assistant. Therefore, there are no instances of communication breakdown to classify within the context of the patient interacting with Alexa.\n\nGiven that the SOAP note does not address or mention any of these

In [16]:
from docx import Document
output_folder="C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns evaluation-final gen"
os.makedirs(output_folder)
for i in range(len(single_evaluation_list)):
    doc=Document()
    doc.add_paragraph(final_evaluation_list[i])
    filename=f"{conversation_names[i]}.docx"
    filepath = os.path.join(output_folder, filename)
    doc.save(filepath)
    print(f"Created: {filepath}")

Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns evaluation-final gen\07-17-2024 L.V.G Baseline.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns evaluation-final gen\07-18-2024 (Week 1) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns evaluation-final gen\07-25-2024 (Week 2) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns evaluation-final gen\08-01-2024 (Week 3) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns evaluation-final gen\08-08-2024 (Week 4) LVG User Needs Assessment.docx.docx
Created: C:/Users/rahul/Desktop/SOAPNote/SOAP_note_generation/gpt4o/communication breakdowns evaluation-final gen\08-22-2024 (Week 6) LVG Debrief Interview.docx.docx
Crea

In [18]:
print(detail_score_generation(demo_soap_note,demo_conversation,conversation_breakdown_list[7]))

Upon reviewing the provided SOAP note, I have conducted a detailed evaluation to determine the presence of the specified breakdown instances. Here is the assessment:

1. **Instance 1:** The SOAP note does not mention any issue related to changing the wake-up word or any similar command failure.

2. **Instance 2:** The SOAP note does not mention any issue related to playing "Sweet Caroline" or any command failure related to music playback.

3. **Instance 3:** The SOAP note does not mention any issue related to playing "Amazing Grace" or the misinterpretation of the "go home" command.

4. **Instance 4:** The SOAP note does not mention any issue related to playing Elvis or the failure to stop advertising.

5. **Instance 5:** The SOAP note does not mention any issue related to the misinterpretation of the "go home" command.

Based on the evaluation, none of the specified breakdown instances are mentioned in the SOAP note. Therefore, the SOAP note receives a score of 0 out of 10, as it does

In [23]:
print(conversation_breakdown_list[7])

1. **Instance 1:**
   - **Conversation:**
     - Speaker 1: "Speaker 4, change your wake up word. It didn't do anything."
     - Speaker 3: "Can you try that one more time?"
     - Speaker 1: "Speaker 4, can you change the wake up word?"
     - Speaker 4: Provides instructions on how to change the wake word.
   - **Category:** @Alexa error
   - **Reason:** The initial command to change the wake-up word did not result in any action or response from Alexa, indicating a potential speech recognition error or other issue.

2. **Instance 2:**
   - **Conversation:**
     - Speaker 1: "Speaker 4, play sweet Caroline."
     - Speaker 4: "Sweet Caroline and similar song on Amazon Music."
     - Speaker 1: "Speaker 4, raise the volume. Speaker 4 stop. Pause. Speaker 4, go back. Go back to the beginning. Speaker 4, play sweet Caroline."
     - Speaker 4: "Sweet Caroline and similar songs on Amazon Music."
   - **Category:** @Alexa error
   - **Reason:** The command to go back to the beginning of t

In [31]:
from pypdf import PdfReader
MA_SOAP_NOTES=[]
path='C:/Users/rahul/Desktop/SOAPNote/dataset/ground_truth_soap_notes/MA'

for i in os.listdir('C:/Users/rahul/Desktop/SOAPNote/dataset/ground_truth_soap_notes/MA'):
    soap_note_ma=""
    file_name=path+'/'+i
    reader=PdfReader(file_name)
    if 'Week_1' not in file_name:
        for j in range(0,len(reader.pages)):
            page=reader.pages[j]
            text=page.extract_text()
            text=text.replace('\n','')
            soap_note_ma+=text
    else:
        for j in range(1,len(reader.pages)):
            page=reader.pages[j]
            text=page.extract_text()
            text=text.replace('\n','')
            soap_note_ma+=text
    MA_SOAP_NOTES.append(soap_note_ma)
    


In [32]:
MA_SOAP_NOTES

["SOAP  Notes  #1:  M.A.  Week  1  (30-40  minutes)  Subjective  (S):   The  participant  attended  the  virtual  session  with  his  mother.  M.A.’s  parents  are  his  primary caregivers who have been helping with his ADLs since his TBI causing blindness along with cognitive communication deficits. M.A. was pleasant and cooperative with the interview questions for the user needs assessment and willing to participate in the tasks for training session. The mother elaborated on M.A’s responses to provide more details for each of the interview questions.  Objective  (O):   User  needs  assessment:  Interview  style  Q  and  A  to  know  about  the  participant’s  needs related to scheduling and reminders was completed.  -  Technology  used  for  scheduling  and  reminders:  Siri  on  iWatch  and  iPhone.  Mother helps with scheduling appointments -  Caregiver  dependent  for  setting  up  appointments  and  reminders   -  Needs  reminders  for  regular  therapy  appointments  at  BRAIN  

In [36]:
RT_SOAP_NOTES=[]
file_name="C:/Users/rahul/Desktop/SOAPNote/dataset/ground_truth_soap_notes/RT/Clinician - RT SOAP Notes.pdf"
reader=PdfReader(file_name)
for i in range(0,len(reader.pages)):
    soap_note_ra=""
    page=reader.pages[i]
    text=page.extract_text()
    text=text.replace('\n','')
    soap_note_ra+=text
    RT_SOAP_NOTES.append(soap_note_ra)

In [37]:
RT_SOAP_NOTES


['SOAP  NOTE:  RT  WEEK  1   GOAL:  Pt  will  utilize  concise  voice  commands  in  80%  of  trials  to  set  an  alarm/timer/reminder  on  the Alexa device given moderate verbal cues.  S:  RT  arrived  to  the  session  on  time,  motivated  and  ready  to  participate  in  the  training.   O   1.  User  Needs:  Pt  identified  two  technology  apps  or  tools  (i.e.,  Apple  Calendar  on  iPhone app, physical wall calendar) she currently uses for scheduling/reminders. Pt identified three scenarios where an alarm/reminder is essential to complete ADLs. Pt identified three appointments in her daily routine that require consistent scheduling and reminders. Pt identified seven scenarios relevant to personal ADLs and weekly routines that she wants to set a timer/reminder/notification for.  2.  VAT  Session:  Pt  utilized  voice  commands  to  set  timers  on  the  Alexa  device  in  3/3  trials in a 1:1 setting with the clinician given an initial model. In a group setting, pt executed 1/

In [ ]:
import textgrad as tg
